# Etude de cas : Utilisation de neo4j avec Python

Dans cette étude de cas, on considère le graphe ayant la structure suivante :


<img src='struct.png' width='40%'>

Ce graphe présente plusieurs enseignant assurant des cours. Ces derniers nécessitent d'autres cours comme pré-requis. <br>
Dans la structure précédente :
<ol>
    <li>Un professeur est représenté par un noeud ayant comme label <b>Prof</b> et possédant deux pripriétés <b>name</b> et <b>city</b> </li>
        <li>Un cours est représenté par un noeud ayant comme label <b>Cours</b> et possédant une seule pripriété <b> label </b> </li>
            <li>La relation entre un professeur et un cours possède le type <b>enseigne</b></li>
            La relation entre un cours et un autre cours possède le type <b>necessite</b>
</ol>

<img src='Exemple1.png' width='60%'>

### Import modules nécessaires

In [165]:
# installer neo4j
#pip install neo4j

In [166]:
from neo4j import GraphDatabase
import pandas as pd

### Connexion à la base de données neo4j

In [167]:
url = "bolt://localhost:7687"
user = "neo4j"
password = "pwd" 

In [168]:
driver = GraphDatabase.driver(url, auth=(user, password))
neo4j = driver.session()

### Création du graphe précédent

La création du graphe peut se faire simplement dans neo4j. On peut aussi le faire à partir de Python.

Dans la suite, on commence par vider la base de données avant de créer les noeuds et les relations, puis on créé notre graphe avec le script ci-dessous :

In [169]:
neo4j.run("match(n) detach delete n")
neo4j.run("""
    create (madani:Prof{name:'Madani', city:'El Jadida'})
    create (elkafi:Prof{name:'El Kafi', city:'Casablanca'})
    create (riffi:Prof{name:'Riffi', city:'Rabat'})
    create (silkan:Prof{name:'Silkan', city:'El Jadida'})
    create (arch:Cours{label:'Architecture'})
    create (algo:Cours{label:'Algorithmique'})
    create (c:Cours{label:'Langage C'})
    create (cpp:Cours{label:'C++'})
    create (bdr:Cours{label:'BDR'})
    create (web:Cours{label:'Prog Web'})
    create (uml:Cours{label:'UML'})
    create (algo)-[:necessite]->(arch)
    create (c)-[:necessite]->(algo)
    create (cpp)-[:necessite]->(c)
    create (bdr)-[:necessite]->(algo)
    create (web)-[:necessite]->(algo)
    create (web)-[:necessite]->(bdr)
    create (uml)-[:necessite]->(bdr)
    create (riffi)-[:enseigne]->(algo)
    create (riffi)-[:enseigne]->(bdr)
    create (madani)-[:enseigne]->(uml)
    create (elkafi)-[:enseigne]->(arch)
    create (silkan)-[:enseigne]->(cpp)
    create (madani)-[:enseigne]->(c)
""")

Pour vérifier la création du graphe, on affiche le nombre d'éléments (noeuds et relations) crées en exécutant le script suivant :

In [170]:
result = neo4j.run("match(n) return count(n)")
print(result.single())

<Record count(n)=11>


### Liste de tous les éléments du graphe (noeuds et relations)

On aura besoin d'afficher les noeuds et les relations du graphe de la base de données pour avoir une idée sur les différents éléments à manipuler

In [171]:
result = neo4j.run("""
MATCH (n) return n as Element
""")
df = pd.DataFrame(result.data())
df

,Element
0,"{'city': 'El Jadida', 'name': 'Madani'}"
1,"{'city': 'Casablanca', 'name': 'El Kafi'}"
2,"{'city': 'Rabat', 'name': 'Riffi'}"
3,"{'city': 'El Jadida', 'name': 'Silkan'}"
4,{'label': 'Architecture'}
5,{'label': 'Algorithmique'}
6,{'label': 'Langage C'}
7,{'label': 'C++'}
8,{'label': 'BDR'}
9,{'label': 'Prog Web'}


Une autre façon qui consite à séparer les noeuds des relations

In [172]:
result = neo4j.run("""
MATCH (n)-[r]->(m) return n as StartNode, r as Relation, m as EndNode
""")
df = pd.DataFrame(result.data())
df

,StartNode,Relation,EndNode
0,"{'city': 'El Jadida', 'name': 'Madani'}","({'city': 'El Jadida', 'name': 'Madani'}, ense...",{'label': 'Langage C'}
1,"{'city': 'El Jadida', 'name': 'Madani'}","({'city': 'El Jadida', 'name': 'Madani'}, ense...",{'label': 'UML'}
2,"{'city': 'Casablanca', 'name': 'El Kafi'}","({'city': 'Casablanca', 'name': 'El Kafi'}, en...",{'label': 'Architecture'}
3,"{'city': 'Rabat', 'name': 'Riffi'}","({'city': 'Rabat', 'name': 'Riffi'}, enseigne,...",{'label': 'BDR'}
4,"{'city': 'Rabat', 'name': 'Riffi'}","({'city': 'Rabat', 'name': 'Riffi'}, enseigne,...",{'label': 'Algorithmique'}
5,"{'city': 'El Jadida', 'name': 'Silkan'}","({'city': 'El Jadida', 'name': 'Silkan'}, ense...",{'label': 'C++'}
6,{'label': 'Algorithmique'},"({'label': 'Algorithmique'}, necessite, {'labe...",{'label': 'Architecture'}
7,{'label': 'Langage C'},"({'label': 'Langage C'}, necessite, {'label': ...",{'label': 'Algorithmique'}
8,{'label': 'C++'},"({'label': 'C++'}, necessite, {'label': 'Langa...",{'label': 'Langage C'}
9,{'label': 'BDR'},"({'label': 'BDR'}, necessite, {'label': 'Algor...",{'label': 'Algorithmique'}


Une autre façon plus ergonomique :

In [173]:
result = neo4j.run("""
MATCH (n)-[r]->(m) return n.name as StartNode, type(r) as Relation, m.label as EndNode
""")
df = pd.DataFrame(result.data())
df

,StartNode,Relation,EndNode
0,Madani,enseigne,Langage C
1,Madani,enseigne,UML
2,El Kafi,enseigne,Architecture
3,Riffi,enseigne,BDR
4,Riffi,enseigne,Algorithmique
5,Silkan,enseigne,C++
6,None,necessite,Architecture
7,None,necessite,Algorithmique
8,None,necessite,Langage C
9,None,necessite,Algorithmique


### Liste de tous les professeurs

On peut afficher les données sous forme d'objets JSON

In [174]:
result = neo4j.run("""
MATCH (n:Prof) return n as Professeur
""")
df = pd.DataFrame(result.data())
df

,Professeur
0,"{'city': 'El Jadida', 'name': 'Madani'}"
1,"{'city': 'Casablanca', 'name': 'El Kafi'}"
2,"{'city': 'Rabat', 'name': 'Riffi'}"
3,"{'city': 'El Jadida', 'name': 'Silkan'}"


On peut aussi afficher les informations de manière séparée, par exemple le nom et la ville de chaque professeur

In [175]:
result = neo4j.run("""
MATCH (n:Prof) return n.name as Name, n.city as City
""")
df = pd.DataFrame(result.data())
df

,Name,City
0,Madani,El Jadida
1,El Kafi,Casablanca
2,Riffi,Rabat
3,Silkan,El Jadida


Comme on peut projeter sur une information donnée. Ici, on est intéressé seulement le nom de chaque professeur.

In [176]:
result = neo4j.run("""
MATCH (n:Prof) return n.name as Name
""")
df = pd.DataFrame(result.data())
df

,Name
0,Madani
1,El Kafi
2,Riffi
3,Silkan


### Liste de tous les cours

In [177]:
result = neo4j.run("""
    match (n:Cours) return n as Cours
""")
df = pd.DataFrame(result.data())
df

,Cours
0,{'label': 'Architecture'}
1,{'label': 'Algorithmique'}
2,{'label': 'Langage C'}
3,{'label': 'C++'}
4,{'label': 'BDR'}
5,{'label': 'Prog Web'}
6,{'label': 'UML'}


In [178]:
result = neo4j.run("""
    match (n:Cours) return n.label as Label
""")
df = pd.DataFrame(result.data())
df

,Label
0,Architecture
1,Algorithmique
2,Langage C
3,C++
4,BDR
5,Prog Web
6,UML


### Les cours enseignés par 'Madani'

In [179]:
result = neo4j.run("""
    match(:Prof{name:'Madani'})-[:enseigne]->(cours)
    return cours.label as cours
""")
df = pd.DataFrame(result.data())
df

,cours
0,Langage C
1,UML


Une autre syntaxe qui ressemble à celle de SQL (bases de données relationnelle)

In [180]:
result = neo4j.run("""
    match(p)-[:enseigne]->(cours)
    where p.name='Madani'
    return cours.label as cours
""")
df = pd.DataFrame(result.data())
df

,cours
0,Langage C
1,UML


### Les professeurs assurant le cours de UML

In [181]:
result = neo4j.run("""
    match (prof)-[:enseigne]->(:Cours{label:'UML'})
    return prof.name as name
""")
df = pd.DataFrame(result.data())
df

,name
0,Madani


### Les pre-requis de module C++

Dans le premier exemple, on affiche le cours directement lié à C++

In [182]:
result = neo4j.run("""
    match (cours)<-[:necessite]-(:Cours{label:'C++'})
    return cours.label as prerequis
""")
df = pd.DataFrame(result.data())
df

,prerequis
0,Langage C


Par contre ici, on désire afficher tous les cours constituant un pré-requis pour le langage C++. Pour cela, on doit récupérer tous les chemins (paths) partant du cours C++

In [183]:
result = neo4j.run("""
    match path = (cours)<-[*]-(:Cours{label:'C++'})
    return cours.label as prerequis
""")
df = pd.DataFrame(result.data())
df

,prerequis
0,Langage C
1,Algorithmique
2,Architecture


## Liste des professeurs par noms décroissants

In [184]:
result = neo4j.run("""
    match(p:Prof)
    return p.name as name
    order by name desc
    //order by name
""")
df = pd.DataFrame(result.data())
df

,name
0,Silkan
1,Riffi
2,Madani
3,El Kafi


## Liste des 2 premiers professeurs par noms décroissants

In [185]:
result = neo4j.run("""
    match(p:Prof)
    return p.name as name
    order by name desc
    limit 2
""")
df = pd.DataFrame(result.data())
df

,name
0,Silkan
1,Riffi


## Liste des 2 premiers professeurs par noms décroissants, en commençant par le 3ème

In [186]:
result = neo4j.run("""
    match(p:Prof)
    return p.name as name
    order by name desc
    skip 2
    limit 2
""")
df = pd.DataFrame(result.data())
df

,name
0,Madani
1,El Kafi


### Ajouter la proprité coefficient aux noeuds Cours

In [187]:
neo4j.run("""
    match(c:Cours{label:'Architecture'})
    set c.coeff=1.5
    return c
""")
neo4j.run("""
    match(c:Cours{label:'Algorithmique'})
    set c.coeff=1.5
    return c
""")
neo4j.run("""
    match(c:Cours{label:'Langage C'})
    set c.coeff=2
    return c
""")
neo4j.run("""
    match(c:Cours{label:'Prog Web'})
    set c.coeff=3
    return c
""")
neo4j.run("""
    match(c:Cours{label:'C++'})
    set c.coeff=4
    return c
""")
neo4j.run("""
    match(c:Cours{label:'BDR'})
    set c.coeff=4
    return c
""")
neo4j.run("""
    match(c:Cours{label:'UML'})
    set c.coeff=5
    return c
""")

In [188]:
result = neo4j.run("""
    match(c:Cours)
    return c.label, c.coeff
""")
df = pd.DataFrame(result.data())
df

,c.label,c.coeff
0,Architecture,1.5
1,Algorithmique,1.5
2,Langage C,2.0
3,C++,4.0
4,BDR,4.0
5,Prog Web,3.0
6,UML,5.0


### Modifier la valeur de la propriété coefficient du cours 'Prog Web'

In [189]:
neo4j.run("""
    match(c:Cours{label:'Prog Web'})
    set c.coeff=3.5
    return c
""")
result = neo4j.run("""
    match(c:Cours)
    return c.label, c.coeff
""")
df = pd.DataFrame(result.data())
df

,c.label,c.coeff
0,Architecture,1.5
1,Algorithmique,1.5
2,Langage C,2.0
3,C++,4.0
4,BDR,4.0
5,Prog Web,3.5
6,UML,5.0


### Supprimer la propriété coefficient des cours 'Architecture' et 'Algorithmique'

La première méthode est d'utiliser <b>set node.propriété = null </b>:

In [190]:
neo4j.run("""
    match(c:Cours{label:'Architecture'})
    set c.coeff=null
    return c
""")
result = neo4j.run("""
    match(c:Cours)
    return c.label, c.coeff
""")
df = pd.DataFrame(result.data())
df

,c.label,c.coeff
0,Architecture,NaN
1,Algorithmique,1.5
2,Langage C,2.0
3,C++,4.0
4,BDR,4.0
5,Prog Web,3.5
6,UML,5.0


La deuxième méthode est d'utiliser <b>remove node.propriété</b>:

In [191]:
neo4j.run("""
    match(c:Cours{label:'Algorithmique'})
    remove c.coeff
    return c
""")
result = neo4j.run("""
    match(c:Cours)
    return c.label, c.coeff
""")
df = pd.DataFrame(result.data())
df

,c.label,c.coeff
0,Architecture,NaN
1,Algorithmique,NaN
2,Langage C,2.0
3,C++,4.0
4,BDR,4.0
5,Prog Web,3.5
6,UML,5.0


### Ajouter le label 'Dir_Lab' au noeud 'Madani'

In [193]:
neo4j.run("""
    match(c:Prof{name:'Madani'})
    set c:Dir_Lab
""")
result = neo4j.run("""
    match(c:Prof)
    return c.name as Name, labels(c) as Labels
""")
df = pd.DataFrame(result.data())
df

,Name,Labels
0,Madani,"[Prof, Dir_Lab]"
1,El Kafi,[Prof]
2,Riffi,[Prof]
3,Silkan,[Prof]
